In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/demo_anomalies_false.csv')

In [ ]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,karnataka,belgaum,591118,27,56
1,01-03-2025,karnataka,bengaluru,560072,92,438
2,01-03-2025,tripura,dhalai,799285,18,517
3,01-03-2025,karnataka,mysore,571121,10,104
4,01-03-2025,delhi,central delhi,110055,109,1458


In [ ]:
df.shape

(273807, 6)

In [ ]:
pip install rapidfuzz jellyfish

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 26.0 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import pandas as pd
from rapidfuzz import process, fuzz
import jellyfish

with open("/content/state_district_reference.json", "r") as f:
    ref = json.load(f)

state_districts = {
    state: data["districts"]
    for state, data in ref.items()
}

def enhanced_similarity(a, b, **kwargs):
    return max(
        fuzz.token_set_ratio(a, b),
        fuzz.partial_ratio(a, b),
        fuzz.ratio(a, b),
        jellyfish.jaro_winkler_similarity(a, b) * 100
    )

def token_or_substring_ok(a, b):
    a = a.lower()
    b = b.lower()

    a_tokens = a.split()
    b_tokens = b.split()

    # token overlap (multi-word safety)
    if set(a_tokens) & set(b_tokens):
        return True

    # substring
    if a in b or b in a:
        return True

    #  single-word prefix rejection
    if len(a_tokens) == 1 and len(b_tokens) == 1:
        if a[:4] != b[:4]:
            return False

    #  character overlap
    a_chars = set(a)
    b_chars = set(b)
    overlap_ratio = len(a_chars & b_chars) / min(len(a_chars), len(b_chars))

    return overlap_ratio >= 0.6



def best_district_match(state, district, threshold=80, min_gap=12):
    if pd.isna(state) or pd.isna(district):
        return np.nan

    if state not in state_districts:
        return np.nan

    matches = process.extract(
        district,
        state_districts[state],
        scorer=enhanced_similarity,
        limit=2
    )

    if not matches:
        return np.nan

    best_name, best_score, _ = matches[0]

    if best_score < threshold:
        return np.nan

    if len(matches) > 1:
        if (best_score - matches[1][1]) < min_gap:
            return np.nan

    if not token_or_substring_ok(district, best_name):
        return np.nan

    return best_name



df["district_new"] = df.apply(
    lambda x: best_district_match(x["state"], x["district"]),
    axis=1
)


In [ ]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,district_new
0,01-03-2025,karnataka,belgaum,591118,27,56,NaN
1,01-03-2025,karnataka,bengaluru,560072,92,438,NaN
2,01-03-2025,tripura,dhalai,799285,18,517,dhalai
3,01-03-2025,karnataka,mysore,571121,10,104,NaN
4,01-03-2025,delhi,central delhi,110055,109,1458,central


In [ ]:
#Saving Cleaned Data
df_cleaned = df[~df['district_new'].isna()]

In [ ]:
df_cleaned.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,district_new
2,01-03-2025,tripura,dhalai,799285,18,517,dhalai
4,01-03-2025,delhi,central delhi,110055,109,1458,central
8,01-03-2025,gujarat,sabarkantha,383355,22,123,sabar kantha
10,01-03-2025,gujarat,surendra nagar,363427,31,272,surendranagar
13,01-03-2025,madhya pradesh,khargone,450551,12,90,khargone (west nimar)


In [ ]:
df_cleaned['district'] = df_cleaned['district_new']

/tmp/ipython-input-1763447331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['district'] = df_cleaned['district_new']


In [ ]:
df_cleaned.drop(columns=['district_new'], inplace=True)


/tmp/ipython-input-3414518825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.drop(columns=['district_new'], inplace=True)


In [ ]:
df_cleaned.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
2,01-03-2025,tripura,dhalai,799285,18,517
4,01-03-2025,delhi,central,110055,109,1458
8,01-03-2025,gujarat,sabar kantha,383355,22,123
10,01-03-2025,gujarat,surendranagar,363427,31,272
13,01-03-2025,madhya pradesh,khargone (west nimar),450551,12,90


In [ ]:
df_cleaned.shape

(84801, 6)

In [ ]:
df_cleaned.to_csv('demo_anomalies_anomalies_clean.csv',index=False)

In [ ]:
# Saving False Data
df_false = df[df['district_new'].isna()]

In [ ]:
df_false.shape

(189006, 7)

In [ ]:
df_false.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,district_new
0,01-03-2025,karnataka,belgaum,591118,27,56,NaN
1,01-03-2025,karnataka,bengaluru,560072,92,438,NaN
3,01-03-2025,karnataka,mysore,571121,10,104,NaN
5,01-03-2025,karnataka,bengaluru,560045,136,892,NaN
6,01-03-2025,karnataka,belgaum,591232,65,116,NaN


In [ ]:
#simple check
df_false[~df_false['district_new'].isna()]

,date,state,district,pincode,demo_age_5_17,demo_age_17_,district_new


In [ ]:
df_false.to_csv('demo_anomalies_anomalies_false.csv',index=False)